<a href="https://colab.research.google.com/github/yckamra/MovieLens25m-Recommender/blob/main/MovieLens_Collaborative_Filtering_vFinal_no_progress_bars.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print("Installing project dependencies...")
!pip install numpy pandas scipy implicit scikit-learn # Basic libraries we want
!pip install --upgrade "implicit==0.7.2" "scipy==1.10.1"
#!pip install torch
print("Dependencies installed successfully.")

Installing project dependencies...
Dependencies installed successfully.


In [2]:
from google.colab import auth
import zipfile
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import implicit
#import torch
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import coo_matrix, csr_matrix # Compressed Sparse Row
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, ndcg_at_k
from google.cloud import storage
import pickle

In [3]:
auth.authenticate_user()

!gcloud config set project movielens-recommender-451017

!gsutil cp gs://movielens-data/movielens_data.zip /content/

Updated property [core/project].
Copying gs://movielens-data/movielens_data.zip...
==> NOTE: You are downloading one or more large file(s), which would
run significantly faster if you enabled sliced object downloads. This
feature is enabled by default but requires that compiled crcmod be
installed (see "gsutil help crcmod").

| [1 files][257.5 MiB/257.5 MiB]                                                
Operation completed over 1 objects/257.5 MiB.                                    


Dataset can be found at: https://www.kaggle.com/datasets/garymk/movielens-25m-dataset

In [4]:
!ls -lh /content/

total 258M
-rw-r--r-- 1 root root 258M Jun 29 23:17 movielens_data.zip
drwxr-xr-x 1 root root 4.0K Jun 26 13:35 sample_data


In [5]:
zip_path = "/content/movielens_data.zip"  # Change to your actual zip file name
extract_to = "/content/data/"  # Where to extract files

# Create directory if it doesn't exist
os.makedirs(extract_to, exist_ok=True)

# Unzip the file
with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall(extract_to)

print("Extraction complete! Files are in:", extract_to)
!ls -lh /content/data/ml-25m

Extraction complete! Files are in: /content/data/
total 1.1G
-rw-r--r-- 1 root root 416M Jun 29 23:18 genome-scores.csv
-rw-r--r-- 1 root root  18K Jun 29 23:18 genome-tags.csv
-rw-r--r-- 1 root root 1.4M Jun 29 23:18 links.csv
-rw-r--r-- 1 root root 2.9M Jun 29 23:18 movies.csv
-rw-r--r-- 1 root root 647M Jun 29 23:18 ratings.csv
-rw-r--r-- 1 root root  11K Jun 29 23:18 README.txt
-rw-r--r-- 1 root root  38M Jun 29 23:18 tags.csv


In [6]:
genome_scores_CSV = "/content/data/ml-25m/genome-scores.csv"
genome_tags_CSV = "/content/data/ml-25m/genome-tags.csv"
links_CSV = "/content/data/ml-25m/links.csv"
movies_CSV = "/content/data/ml-25m/movies.csv"
ratings_CSV = "/content/data/ml-25m/ratings.csv"
tags_CSV = "/content/data/ml-25m/tags.csv"

In [7]:
# Load ratings data
ratings_path = ratings_CSV
ratings_df = pd.read_csv(ratings_path)

print(ratings_df)

# Load movies data
movies_path = movies_CSV
movies_df = pd.read_csv(movies_path)

print(movies_df)

# Load tags data
tags_path = tags_CSV
tags_df = pd.read_csv(tags_path)

print(tags_df)

          userId  movieId  rating   timestamp
0              1      296     5.0  1147880044
1              1      306     3.5  1147868817
2              1      307     5.0  1147868828
3              1      665     5.0  1147878820
4              1      899     3.5  1147868510
...          ...      ...     ...         ...
25000090  162541    50872     4.5  1240953372
25000091  162541    55768     2.5  1240951998
25000092  162541    56176     2.0  1240950697
25000093  162541    58559     4.0  1240953434
25000094  162541    63876     5.0  1240952515

[25000095 rows x 4 columns]
       movieId                               title  \
0            1                    Toy Story (1995)   
1            2                      Jumanji (1995)   
2            3             Grumpier Old Men (1995)   
3            4            Waiting to Exhale (1995)   
4            5  Father of the Bride Part II (1995)   
...        ...                                 ...   
62418   209157                           

In [8]:
# Sort by user
ratings_df.sort_values('userId', inplace=True, ignore_index=True)

# Count unique users and movies
num_users = ratings_df["userId"].nunique()
num_movies = ratings_df["movieId"].nunique()

print(f"Number of Unique Users: {num_users}")
print(f"Number of Unique Movies: {num_movies}")

Number of Unique Users: 162541
Number of Unique Movies: 59047


We will be using collaborative filtering for recommendations. A 2D matrix will be created for movie to user ratings with movies on x-axis and users on y-axis. For new users, we ask for a baseline amount of movies to get a starting vector, and then we use cosine similarity to find their similarity to existing users to recommend them movies. With more ratings, comes greater accuracy.

In [9]:
#numpy_matrix = np.zeros((num_users, num_movies))
#print(numpy_matrix)
#print(numpy_matrix.shape)

In order to map IDs of movies and users to the 2D matrix for collaborative filtering, we will use dictionaries to create key-value pairs between ID and index within the matrix.

In [10]:
user_id_map = {}
movie_id_map = {}
user_index = 0
movie_index = 0

# Create user_id and movie_id to 2D matrix indices
for user in ratings_df["userId"]: # Use this for sparse matrix, not userId since users could be removed from dataset
  if user not in user_id_map:
    user_id_map[user] = user_index
    user_index += 1

for movie in ratings_df["movieId"]:
  if movie not in movie_id_map:
    movie_id_map[movie] = movie_index
    movie_index += 1

Notice below that we are populating a dense matrix, so if we have ~25 million cells to populate out of the ~9 billion from our 162,541 x 59047 matrix filled with zeroes, we are dealing with major sparcity problems and wasted memory. So below is just to later enjoy shorter train times and smaller matrices. Going forward we will be using a Compressed Sparse Row matrix. This will help in learning user and movie abstract feature matrices and cosine similarities.

In [11]:
#from IPython.display import clear_output

#iter = 0
#percent_done = 0
#total_rows = len(ratings_df)
# Iterate over the rows of the DataFrame
#for index, row in ratings_df.iterrows():
    #user_index = user_id_map[row["userId"]]
    #movie_index = movie_id_map[row["movieId"]]
    #numpy_matrix[user_index, movie_index] = row["rating"]
    #if iter % int(total_rows / 100) == 0:
      #clear_output(wait=True)
      #print(f"Percent of matrix populated with ratings: {percent_done}%")
      #percent_done += 1
    #iter += 1

In [12]:
#ratings_dataframe = pd.DataFrame(numpy_matrix)
#print(ratings_dataframe.head())

In [13]:
# Comparing the dataframe ratings to the original Kaggle dataset
#print(f"Original Rating: 4.0 Found Rating: {ratings_dataframe.loc[user_id_map[1], movie_id_map[8786]]}")
#print(f"Original Rating: 2.0 Found Rating: {ratings_dataframe.loc[user_id_map[2], movie_id_map[480]]}")
#print(f"Original Rating: 3.5 Found Rating: {ratings_dataframe.loc[user_id_map[3], movie_id_map[1270]]}")
#print(ratings_dataframe.shape)

Here I have created my own arrays for sparse matrix. We will use scipy in order to have more accurate conversions, but this gives us another option and removes the blackbox.

In [14]:
def calculate_data_array(df):  # Numpy array of ratings
  ratings_df = df
  data = ratings_df["rating"].to_numpy()
  print(data)
  return data

def calculate_movie_indices(df):   # Numpy array of movie indices
  ratings_df = df
  indices = ratings_df["movieId"].to_numpy()

  for i in range(len(indices)):
      indices[i] = movie_id_map[int(indices[i])]
  print(indices)
  return indices


def calculate_indptr(df):   # Numpy array of pointers to indices
  ratings_df = df
  num_users = ratings_df["userId"].nunique()
  indptr = np.zeros(num_users + 1, dtype=int)
  last_user = ratings_df["userId"].iloc[0]
  index_in_indptr = 1
  iterator = 0

  for i in range(len(ratings_df["userId"])):
    current_user = ratings_df["userId"].iloc[iterator]
    if current_user != last_user:
      indptr[index_in_indptr] = iterator
      index_in_indptr += 1
      last_user = current_user
    iterator += 1

  indptr[index_in_indptr] = len(ratings_df["userId"]) # This fills up the last indice

  print(indptr)
  return indptr

#data = {'userId': [0, 0, 1, 2, 2, 2, 3]} # For testing
#df = pd.DataFrame(data) # For testing
#calculate_indptr(df) # For testing

In [15]:
# Apply the maps to the DataFrame columns to get the 0-indexed row and column arrays
row_indices = ratings_df['userId'].map(user_id_map).to_numpy()
col_indices = ratings_df['movieId'].map(movie_id_map).to_numpy()
data_values = ratings_df['rating'].to_numpy() # The actual ratings

# Master COO matrix
user_item_matrix_coo = coo_matrix(
    (data_values, (row_indices, col_indices)),
    shape=(num_users, num_movies)
)

# Master CSR matrix
user_item_matrix_csr = user_item_matrix_coo.tocsr()

# Split into train and test sets
#(train_row_indices, test_row_indices,
# train_col_indices, test_col_indices,
# train_data_values, test_data_values) = train_test_split(
#    row_indices,
#    col_indices,
#    data_values,
#    test_size=0.3,
#    random_state=42
#)

#print(train_row_indices)

# Create the train COO Matrix
#train_matrix_coo = coo_matrix(
#    (train_data_values, (train_row_indices, train_col_indices)),
#    shape=(num_users, num_movies)
#)

# Convert to CSR (Compressed Sparse Row) for the training set
#train_matrix_csr = train_matrix_coo.tocsr()

# Create the test COO Matrix
#test_matrix_coo = coo_matrix(
#    (test_data_values, (test_row_indices, test_col_indices)),
#    shape=(num_users, num_movies)
#)

# Convert to CSR (Compressed Sparse Row) for the test set
#test_matrix_csr = test_matrix_coo.tocsr()

# Stack the row, col, value into tuples
interactions = list(zip(row_indices, col_indices, data_values))

# Use train_test_split on the full triplets
train_data, test_data = train_test_split(interactions, test_size=0.3, random_state=42)

# Separate the triplets again for matrix construction
train_rows, train_cols, train_vals = zip(*train_data)
test_rows, test_cols, test_vals = zip(*test_data)

# Build sparse matrices
train_matrix_csr = coo_matrix((train_vals, (train_rows, train_cols)), shape=(num_users, num_movies)).tocsr()
test_matrix_csr = coo_matrix((test_vals, (test_rows, test_cols)), shape=(num_users, num_movies)).tocsr()

Notice below that the indice array is different than our function. This is because although the user indice array is sorted as the main sort, within each user's range--the subsection for each user--has their movie indices sorted as well.

In [16]:
print("Converted to CSR Matrix.")
print("Train CSR Shape:", train_matrix_csr.shape)
print("Train CSR number of examples:", train_matrix_csr.nnz)
print("Train CSR data:", train_matrix_csr.data)
print("Train CSR indices (column indices):", train_matrix_csr.indices)
print("Train CSR indptr (pointers to row starts):", train_matrix_csr.indptr)
print("Test CSR Shape:", test_matrix_csr.shape)
print("Test CSR number of examples:", test_matrix_csr.nnz)
print("Test CSR data:", test_matrix_csr.data)
print("Test CSR indices (column indices):", test_matrix_csr.indices)
print("Test CSR indptr (pointers to row starts):", test_matrix_csr.indptr)

Converted to CSR Matrix.
Train CSR Shape: (162541, 59047)
Train CSR number of examples: 17500066
Train CSR data: [5.  5.  3.5 ... 2.  3.  2.5]
Train CSR indices (column indices): [    0     3     4 ... 12213 12780 14125]
Train CSR indptr (pointers to row starts): [       0       55      188 ... 17499873 17499937 17500066]
Test CSR Shape: (162541, 59047)
Test CSR number of examples: 7500029
Test CSR data: [2.5 4.  4.  ... 4.  4.  3.5]
Test CSR indices (column indices): [   1    2   12 ... 3845 5693 6221]
Test CSR indptr (pointers to row starts): [      0      15      66 ... 7499952 7499976 7500029]


In [17]:
def get_cosine_similarity_userX_to_allUsers(userId, user_id_map, user_item_matrix_csr): # TODO: we need to remove the own user from its similarity array
  user_index = user_id_map[userId]
  target_user_vector = user_item_matrix_csr[user_index : user_index + 1]
  return cosine_similarity(target_user_vector, user_item_matrix_csr)

In [18]:
print(get_cosine_similarity_userX_to_allUsers(1, user_id_map, train_matrix_csr))

[[1.         0.04059364 0.03063487 ... 0.         0.01862104 0.06664523]]


Our model for matrix factorization will be Alternating Least Squares (ALS).

In [ ]:
# Hyperparameters
num_factors = 50       # Number of dimensions to learn
regularization = 0.01  # L2 regularization
num_iterations = 20    # Number of training iterations

# ALS model
matrix_factorization_model = implicit.als.AlternatingLeastSquares(
    factors=num_factors,
    regularization=regularization,
    iterations=num_iterations,
    random_state=42,
    calculate_training_loss=True # Monitors loss
)

# Model will learn confidence of a user's interest in a movie and will not predict scores (0.5-5.0)
# Larger confidence score, the more likely the user will like the movie
matrix_factorization_model.fit(train_matrix_csr)

# Store our user and movie abstract feature (latent) matrices
user_factors = matrix_factorization_model.user_factors # Shape is num_users by num_factors
item_factors = matrix_factorization_model.item_factors # Shape is num_items by num_factors

print("ALS model training complete!")
print(f"Learned User Factors Shape: {user_factors.shape}")
print(f"Learned Movie Factors Shape: {item_factors.shape}")

Model 1:
*   50 factors, 0.01 regularization, 20 num of iterations
*   training loss: 0.00301
*   Precision@20: 0.4536
*   MAP@20: 0.2945
*   NDCG@20: 0.4506

Model 2:
*   100 factors, 0.01 regularization, 20 num of iterations
*   0.00278 loss on train
*   Precision@20: 0.4388
*   MAP@20: 0.2772
*   NDCG@20: 0.4359


In [ ]:
K = 20

# Evaluate
precision = precision_at_k(matrix_factorization_model, train_matrix_csr, test_matrix_csr, K=K)
map_k = mean_average_precision_at_k(matrix_factorization_model, train_matrix_csr, test_matrix_csr, K=K)
ndcg = ndcg_at_k(matrix_factorization_model, train_matrix_csr, test_matrix_csr, K=K)

print(f"Precision@{K}: {precision:.4f}")
print(f"MAP@{K}: {map_k:.4f}")
print(f"NDCG@{K}: {ndcg:.4f}")

In [ ]:
def save_to_gcs(thing_to_save, bucket_name: str, destination_blob_name: str):
    try:
        auth.authenticate_user()
        print("Authenticated to Google Cloud.")
    except Exception as e:
        print(f"Could not authenticate user (may be running outside Colab or already authenticated): {e}")

    client = storage.Client()
    bucket = client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    # Pickle the object into a bytes object
    pickled_data = pickle.dumps(thing_to_save)

    # Upload object to GCS
    blob.upload_from_string(pickled_data, content_type='application/octet-stream')

    print(f"Object successfully saved to gs://{bucket_name}/{destination_blob_name}")

In [ ]:
bucket_name = "movielens-data"

blob_name = "collaborative_filtering/user_id_map.pkl"
save_to_gcs(user_id_map, bucket_name, blob_name)

blob_name = "collaborative_filtering/movie_id_map.pkl"
save_to_gcs(movie_id_map, bucket_name, blob_name)

blob_name = "collaborative_filtering/user_item_matrix_csr.pkl"
save_to_gcs(user_item_matrix_csr, bucket_name, blob_name)

blob_name = "collaborative_filtering/train_matrix_csr.pkl"
save_to_gcs(train_matrix_csr, bucket_name, blob_name)

blob_name = "collaborative_filtering/test_matrix_csr.pkl"
save_to_gcs(test_matrix_csr, bucket_name, blob_name)

blob_name = "collaborative_filtering/matrix_factorization_model.pkl"
save_to_gcs(matrix_factorization_model, bucket_name, blob_name)